In [13]:
from search_in_video.models import Video
from google.cloud import videointelligence_v1p2beta1 as videointelligence
from django.conf import settings
import os

In [10]:
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [15]:
video = Video.objects.get(id=59)

In [16]:
video = video.video
storage_uri = f'gs://{settings.GS_BUCKET_NAME}/{video.name}'
video_client = videointelligence.VideoIntelligenceServiceClient()
features = [videointelligence.enums.Feature.TEXT_DETECTION]
operation = video_client.annotate_video(
    input_uri=storage_uri,
    features=features
)

In [22]:
op = operation

In [33]:
dir(op)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_blocking_poll',
 '_cancel',
 '_completion_lock',
 '_done_callbacks',
 '_done_or_raise',
 '_exception',
 '_invoke_callbacks',
 '_metadata_type',
 '_operation',
 '_polling_thread',
 '_refresh',
 '_refresh_and_update',
 '_result',
 '_result_set',
 '_result_type',
 '_retry',
 '_set_result_from_operation',
 'add_done_callback',
 'cancel',
 'cancelled',
 'deserialize',
 'done',
 'exception',
 'metadata',
 'operation',
 'result',
 'running',
 'set_exception',
 'set_result']

In [39]:
result = op.result()

In [44]:
annotation_result = result.annotation_results[0]

In [47]:
text_annotation = annotation_result.text_annotations[0]

In [61]:
def pp(text, indent=0, end='\n'):
    print('\t'*indent, end='')
    print(text, end=end)
    
pp('test', indent=1)

	test


In [63]:
for text_annotation in annotation_result.text_annotations:
    text = text_annotation.text
    text_segments = text_annotation.segments
    pp(text)
    for text_segment in text_segments:
        start_time = text_segment.segment.start_time_offset
        end_time = text_segment.segment.end_time_offset
        pp('start_time: {}, end_time: {}'.format(
            start_time.seconds + start_time.nanos * 1e-9,
            end_time.seconds + end_time.nanos * 1e-9), indent=1)
        frame = text_segment.frames[0]
        for vertex in frame.rotated_bounding_box.vertices:
            print('Vertex.x: {}, Vertex.y: {}'.format(vertex.x, vertex.y))

jennykang
	start_time: 6.9, end_time: 13.5 / Vertex.x: 0.28125, Vertex.y: 0.3592592477798462
Vertex.x: 0.3229166567325592, Vertex.y: 0.3592592477798462
Vertex.x: 0.3229166567325592, Vertex.y: 0.3759259283542633
Vertex.x: 0.28125, Vertex.y: 0.3759259283542633
__28 x 28 –
	start_time: 14.0, end_time: 14.6 / Vertex.x: 0.25405094027519226, Vertex.y: 0.36851853132247925
Vertex.x: 0.3495319187641144, Vertex.y: 0.3701016306877136
Vertex.x: 0.34917211532592773, Vertex.y: 0.42565420269966125
Vertex.x: 0.2536911368370056, Vertex.y: 0.4240711033344269
	start_time: 20.3, end_time: 30.3 / Vertex.x: 0.25578704476356506, Vertex.y: 0.36851853132247925
Vertex.x: 0.34953194856643677, Vertex.y: 0.3700809180736542
Vertex.x: 0.3491702973842621, Vertex.y: 0.4256334602832794
Vertex.x: 0.255425363779068, Vertex.y: 0.4240710735321045
	start_time: 32.8, end_time: 39.1 / Vertex.x: 0.2517361044883728, Vertex.y: 0.36851853132247925
Vertex.x: 0.3506890833377838, Vertex.y: 0.37016770243644714
Vertex.x: 0.35032740235

In [53]:
text_annotation.text

'jennykang'